In [80]:
web1="https://physics.nist.gov/PhysRefData/XrayMassCoef/tab1.html"
web3="https://physics.nist.gov/PhysRefData/XrayMassCoef/tab3.html"

In [81]:
import pandas as pd
import numpy as np

In [82]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

req = Request(web3)
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "lxml")

links = []
name=[]
for link in soup.findAll('a'):
    links.append('https://physics.nist.gov/PhysRefData/XrayMassCoef/'+link.get('href'))
    name.append(link.get('href'))

#print(links)

In [83]:
elementos=links[1:-6]
nombre=[X.replace('ElemTab/z',"")[:-5] for X in name[1:-6]]

In [84]:
Z=np.array(nombre).astype(np.int16)

In [85]:
xcom_elements={}
for i,address in zip(Z,elementos):
    xcom_elements[i]=address

In [86]:
xcom_elements

{1: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z01.html',
 24: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z24.html',
 47: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z47.html',
 70: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z70.html',
 2: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z02.html',
 25: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z25.html',
 48: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z48.html',
 71: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z71.html',
 3: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z03.html',
 26: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z26.html',
 49: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z49.html',
 72: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z72.html',
 4: 'https://physics.nist.gov/PhysRefData/XrayMassCoef/ElemTab/z04.html',
 27: 'https://physics.nist.go

In [101]:
Material=pd.read_csv('MaterialConstant.csv')

In [102]:
Material=Material.drop('Unnamed: 0',axis=1)

In [103]:
Material.index = np.arange(1, len(Material) + 1)

In [104]:
Material

,Z,Element,Z/A,I,Density
1,H,Hydrogen,0.99212,19.2,0.000084
2,He,Helium,0.49968,41.8,0.000166
3,Li,Lithium,0.43221,40.0,0.534000
4,Be,Beryllium,0.44384,63.7,1.848000
5,B,Boron,0.46245,76.0,2.370000
...,...,...,...,...,...
88,Ra,Radium,0.38934,826.0,5.000000
89,Ac,Actinium,0.39202,841.0,10.070000
90,Th,Thorium,0.38787,847.0,11.720000
91,Pa,Protactinium,0.39388,878.0,15.370000


In [107]:
Material=Material.rename(columns={'Z':'El'})

In [108]:
Material

,El,Element,Z/A,I,Density
1,H,Hydrogen,0.99212,19.2,0.000084
2,He,Helium,0.49968,41.8,0.000166
3,Li,Lithium,0.43221,40.0,0.534000
4,Be,Beryllium,0.44384,63.7,1.848000
5,B,Boron,0.46245,76.0,2.370000
...,...,...,...,...,...
88,Ra,Radium,0.38934,826.0,5.000000
89,Ac,Actinium,0.39202,841.0,10.070000
90,Th,Thorium,0.38787,847.0,11.720000
91,Pa,Protactinium,0.39388,878.0,15.370000


In [109]:
mu=[]
muen=[]
for i,row in Material.iterrows():
    url=xcom_elements[i]
    A=pd.read_html(url, flavor = 'bs4')[0]
    
    if (A[A.eq('Energy').any(1)][2].values[0]=='μ/ρ'):
        A.dropna(subset = [0], inplace=True)
        B=A[[0,2,4]]
    else:
        A.dropna(subset = [2], inplace=True)
        B=A[[2,4,6]]
    B = B[4:]
    B.columns=['Energy','mu','muen']
    B=B.reset_index()
    B.drop('index',inplace=True,axis=1)
    B=B.set_index('Energy')
    mu.append(B.astype(float).to_dict('series')['mu'])
    muen.append(B.astype(float).to_dict('series')['muen'])

In [110]:
Material['mu']=mu

In [111]:
Material['muen']=muen

In [112]:
Material

,El,Element,Z/A,I,Density,mu,muen
1,H,Hydrogen,0.99212,19.2,0.000084,Energy 1.00000E-03 7.21700 1.50000E-03 2...,Energy 1.00000E-03 6.820000 1.50000E-03 ...
2,He,Helium,0.49968,41.8,0.000166,Energy 1.00000E-03 60.84000 1.50000E-03 ...,Energy 1.00000E-03 60.450000 1.50000E-03 ...
3,Li,Lithium,0.43221,40.0,0.534000,Energy 1.00000E-03 233.90000 1.50000E-03 ...,Energy 1.00000E-03 233.500000 1.50000E-03 ...
4,Be,Beryllium,0.44384,63.7,1.848000,Energy 1.00000E-03 604.10000 1.50000E-03 ...,Energy 1.00000E-03 603.500000 1.50000E-03 ...
5,B,Boron,0.46245,76.0,2.370000,Energy 1.00000E-03 1229.00000 1.50000E-03 ...,Energy 1.00000E-03 1228.00000 1.50000E-03 ...
...,...,...,...,...,...,...,...
88,Ra,Radium,0.38934,826.0,5.000000,Energy 1.00000E-03 6201.00000 1.02840E-03 ...,Energy 1.00000E-03 6187.00000 1.02840E-03 ...
89,Ac,Actinium,0.39202,841.0,10.070000,Energy 1.00000E-03 6469.00000 1.03923E-03 ...,Energy 1.00000E-03 6455.00000 1.03923E-03 ...
90,Th,Thorium,0.38787,847.0,11.720000,Energy 1.00000E-03 6614.00000 1.08088E-03 ...,Energy 1.00000E-03 6600.00000 1.08088E-03 ...
91,Pa,Protactinium,0.39388,878.0,15.370000,Energy 1.00000E-03 6530.00000 1.00334E-03 ...,Energy 1.00000E-03 6515.00000 1.00334E-03 ...


In [113]:
Material.to_json('MassAttenuationElements.json',orient='index')

In [29]:
for z,mass in zip(nombre,elementos):
    url=mass
    A=pd.read_html(url, flavor = 'bs4')[0]
    
    if (A[A.eq('Energy').any(1)][2].values[0]=='μ/ρ'):
        A.dropna(subset = [0], inplace=True)
        B=A[[0,2,4]]
    else:
        A.dropna(subset = [2], inplace=True)
        B=A[[2,4,6]]
    B = B[4:]
    B.columns=['Energy','mu','muen']
    B=B.reset_index()
    B.drop('index',inplace=True,axis=1)
    B=B.set_index('Energy')
    masscoefi[z]=B.astype(float).to_dict('series')

NameError: name 'masscoefi' is not defined

In [30]:
url=xcom_elements[1]
A=pd.read_html(url, flavor = 'bs4')[0]
    
if (A[A.eq('Energy').any(1)][2].values[0]=='μ/ρ'):
    A.dropna(subset = [0], inplace=True)
    B=A[[0,2,4]]
else:
    A.dropna(subset = [2], inplace=True)
    B=A[[2,4,6]]
B = B[4:]
B.columns=['Energy','mu','muen']
B=B.reset_index()
B.drop('index',inplace=True,axis=1)
B=B.set_index('Energy')

In [79]:
Material.to_json('MassAttenuationElements.json',orient='index')

In [76]:
import json
with open('masscoefi.json', 'w') as fp:
    json.dump(Material.to_dict('index'), fp)

TypeError: Object of type Series is not JSON serializable